In [2]:
import matplotlib
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

# pre process data module
from ProcessCalcium import ProcessCalcium

# 
# data_dir = '/media/cat/1TB/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'



Autosaving every 180 seconds


In [3]:
###########################################################################
##### PROCESS [CA] DATA (EVENT TRIGGERED MAPS): ROI OR PCA TIME SERIES ####
###########################################################################
'''  Compute calcium activity in ROIs selected (35) for 
     lever pull actiivty
     
     or just use PCA time courses 
'''

# Processing flags
parallel = False  # THIS DOESN"T REALLY WORK TOO MUCH MEMORY...
n_cores = 16

# select animal names
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datase
names = ['IA2']


# window to compute
n_sec_window = 30
lockout_window = 10   # no of seconds of previous movement / lever pull lockout
recompute = True      # overwrite previously generated data

# # save PCA vs. ROI data; if selected, each dataset will be reduced to the PCs rquired to get to the explained var value
pca_etm = True
pca_explained_var_val = 0.95

################################
etm = ProcessCalcium()
etm.main_dir = data_dir
etm.verbose = True

#
etm.random_events_lockout = 3.0  # minimum number of seconds difference between a rewarded pull and a random/control 
etm.n_sec_window = n_sec_window 
etm.recompute = recompute
etm.pca_fixed_comps = 30  # fixed numer of components overrides explained_variance amount above

# 
# etm.remove_shift = True     # don't use shift
etm.pca_n_components = 20
etm.whole_stack = True
etm.feature_name = "whole_stack"

#
etm.export_blue_light_times=False

#
etm.img_rate = 30.0
etm.low_cut = 0.1
etm.high_cut = etm.img_rate/2.-0.01

#
#features = ['left_paw','right_paw','jaw']

# 
etm.feature_quiescence = 3    # number of seconds of no-movement prior to feature movement ;not applicable to code-04
                                # this value is computed in generate_movements_quiescence_periods notebook;
# 
#etm.skip_to = 'Mar14_'  # flag to pickup reprocessing at some point; only used if overwrite flag is set to True and need to partially restart
etm.skip_to = None  # 

# 
etm.sessions = 'IA2pm_Apr22_Week2_30Hz' #"all"

#

# 
etm.generate_etm(names,
                 n_sec_window,
                 lockout_window,
                 recompute,
                 pca_etm,
                 pca_explained_var_val,
                 parallel,
                 n_cores,
                 )                

# 

100%|██████████| 44/44 [00:00<00:00, 341126.39it/s]

PROCESSING:  IA2


In [2]:
#############################################
########## STM MOTIFFS WITH MASKS ###########
#############################################

spatial_r = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_code_04_random_ROItimeCourses_30sec_pca_0.95_spatial.npy')
temporal_r = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_code_04_random_ROItimeCourses_30sec_pca_0.95.npy')

spatial = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95_spatial.npy')
temporal = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')

print (spatial.shape, temporal.shape)

#####################################################
data = np.matmul(temporal.transpose(0,2,1), spatial)
print (data.shape)

data_r = np.matmul(temporal_r.transpose(0,2,1), spatial_r)
print (data_r.shape)

(6, 16384) (106, 6, 1801)
(106, 1801, 16384)
(106, 1801, 16384)


In [86]:
#####################################################
#####################################################
#####################################################

# 
data_in = data_r
#data_in = data

mask = np.int32(np.loadtxt('/media/cat/4TBSSD/yuki/IJ2/genericmask.txt'))
contours = np.load('/media/cat/4TBSSD/yuki/meta_allenmap/contours_fig1.npy')
print (mask)
mask2 = np.ones((128,128),'float32')
for k in range(mask.shape[0]):
    #print (mask[k])
    mask2[mask[k][0],mask[k][1]]=np.nan


data_in = data_in.reshape(106, 1801, 128, 128)
start = 300
end = 1020
bin_width = 60

trial_ids = [2,33,44,23,65,76]
# 
for trial_id in trial_ids:
    img = []
    bin_width = 60
    for k in range(start, end, bin_width):
        temp = data_in[trial_id, k:k+bin_width].mean(0)
        temp = np.roll(temp,-2,axis=0)
        temp *= mask2

        temp = temp*contours
        img.append(temp)

    img = np.hstack(img)
    print (img.shape)

    vmin = -0.10/2.
    vmax = 0.10/2.
    
    
    plt.imshow(img, vmin=vmin, vmax=vmax,
               #aspect='auto',
               interpolation='none')
    plt.xticks([])
    plt.yticks([])


    vmin = np.nanmin(img)
    vmax = np.nanmax(img)
    print ("vmin, vmax: ", vmin, vmax)
    plt.savefig('/home/cat/'+str(trial_id)+'.svg',dpi=1200)
    plt.close()
    
    

[[  0   0]
 [  0   1]
 [  0   2]
 ...
 [127 125]
 [127 126]
 [127 127]]
(128, 1536)
vmin, vmax:  -0.07974244956345516 0.06392255067824296
(128, 1536)
vmin, vmax:  -0.06832434285464498 0.07076764024306223
(128, 1536)
vmin, vmax:  -0.06283431901104171 0.07496127550780378
(128, 1536)
vmin, vmax:  -0.06905129998572893 0.06723553408577641
(128, 1536)
vmin, vmax:  -0.07724548819142402 0.07634448606211491
(128, 1536)
vmin, vmax:  -0.06239139806366803 0.07082611885323893


In [87]:
#####################################################
#####################################################
#####################################################


mask = np.int32(np.loadtxt('/media/cat/4TBSSD/yuki/IJ2/genericmask.txt'))

print (mask)
mask2 = np.ones((128,128),'float32')
for k in range(mask.shape[0]):
    #print (mask[k])
    mask2[mask[k][0],mask[k][1]]=np.nan

# 
img_all = []
for trial_id in range(data.shape[0]): 
    img=[]
    for k in range(start, end, bin_width):
        temp = data_in[trial_id, k:k+bin_width].mean(0)
        #print (temp.shape, mask2.shape)
        temp *= mask2
        temp *= contours
        img.append(temp)

    img_all.append(np.hstack(img))


img = np.mean(img_all, axis=0)


vmin = -0.035
vmax = 0.035

    
cbar = plt.imshow(img, 
                  vmin=vmin, 
                  vmax=vmax,
           #aspect='auto',
           interpolation='none')
plt.xticks([])
plt.yticks([])
#plt.colorbar()

vmin = np.nanmin(img_all)
vmax = np.nanmax(img_all)
print ("vmin, vmax: ", vmin, vmax)

plt.savefig('/home/cat/all.svg',dpi=1200)
plt.close()

[[  0   0]
 [  0   1]
 [  0   2]
 ...
 [127 125]
 [127 126]
 [127 127]]
vmin, vmax:  -0.09494960650938924 0.09430146576089161


In [33]:
# ########################################################
# ########## CODE TO MAKE CONTOUR MASK PART 1 ############
# ########################################################

# idx = np.where(edges1==1)
# edges1[idx]=np.nan
# edges1 = edges1*np.roll(mask2,2,axis=0)
# # plt.imshow(edges1)
# # plt.show()

# print (np.unique(edges1))

# mask3 = edges1.copy()

# idx = np.where(mask3==1)
# mask3[idx]=np.nan
# idx = np.where(mask3==0)
# mask3[idx]=1
# plt.imshow(mask3)
# print ("unique: ", np.unique(mask3))
# plt.show()




[ 0. nan nan ... nan nan nan]
unique:  [ 1. nan nan ... nan nan nan]


In [76]:
# ########################################################
# ########## CODE TO MAKE CONTOUR MASK PART 2 ############
# ########################################################

# import matplotlib.image as mpimg

# coords = []
# def onclick(event): 
#     print("button=%d, x=%d, y=%d, xdata=%f, ydata=%f" % ( 
#          event.button, event.x, event.y, event.xdata, event.ydata)) 
#     coords.append([event.xdata,event.ydata])
    
    
# for k in range(len(coords_saved_nans)):
#     try:
#         mask3[coords_saved_nans[k][1],coords_saved_nans[k][0]]=np.nan
#     #plt.scatter(coords_saved[k][0],coords_saved[k][1],s=5,c='red')
#     except:
#         pass
    
# for k in range(len(coords_saved)):
#     mask3[coords_saved[k][1],coords_saved[k][0]]=1
#     #plt.scatter(coords_saved[k][0],coords_saved[k][1],s=5,c='red')


    
# ax = plt.imshow(mask3)
# fig = ax.get_figure()

# cid = fig.canvas.mpl_connect('button_press_event', onclick) 

# plt.show()

# print (coords)

# np.save('//media/cat/4TBSSD/yuki/meta_allenmap/contours_fig1.npy',mask3)

[]


In [75]:
# ########################################################
# ########## CODE TO MAKE CONTOUR MASK PART 3 ############
# ########################################################

# # either turn pixels on or off
# if True:
#     coords_saved_nans = np.vstack((coords_saved_nans,np.int32(coords)))
# else:
#     coords_saved = np.vstack((coords_saved,np.int32(coords)))


In [4]:
# ########################################################
# ########## CODE TO MAKE CONTOUR MASK PART 0 ############
# ########################################################

# image = np.load('/media/cat/4TBSSD/yuki/meta_allenmap/maskwarp.npy')

# #image[:,65]=55

# import numpy as np
# import matplotlib.pyplot as plt
# from scipy import ndimage as ndi
# from skimage.util import random_noise
# from skimage import feature


# # Generate noisy image of a square
# # image = np.zeros((128, 128), dtype=float)
# # image[32:-32, 32:-32] = 1

# # image = ndi.rotate(image, 15, mode='constant')
# # image = ndi.gaussian_filter(image, 4)
# # image = random_noise(image, mode='speckle', mean=0.1)

# # Compute the Canny filter for two values of sigma
# edges1 = feature.canny(image, sigma=1)
# edges2 = feature.canny(image, sigma=3)

# # display results
# fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(8, 3))

# ax[0].imshow(image, cmap='gray')
# ax[0].set_title('noisy image', fontsize=20)

# ax[1].imshow(edges1, cmap='gray')
# ax[1].set_title(r'Canny filter, $\sigma=1$', fontsize=20)

# ax[2].imshow(edges2, cmap='gray')
# ax[2].set_title(r'Canny filter, $\sigma=3$', fontsize=20)

# for a in ax:
#     a.axis('off')

# fig.tight_layout()



# plt.show()


[[  0   0]
 [  0   1]
 [  0   2]
 ...
 [127 125]
 [127 126]
 [127 127]]
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128

(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128)

(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128) (128, 128)
(128, 128)

In [70]:
###################################################
###################################################
###################################################
#
mask = np.int32(np.loadtxt('/media/cat/4TBSSD/yuki/IJ2/genericmask.txt'))

print (mask)
mask2 = np.ones((128,128),'float32')
for k in range(mask.shape[0]):
    #print (mask[k])
    mask2[mask[k][0],mask[k][1]]=np.nan

# 
data_sets = [data,data_r]
variances =[]
for ctr,data_in in enumerate(data_sets):
    #data_in = data
    #data_in = data
    data_in = data_in.reshape(106, 1801, 128, 128)

    img_all = []
    for trial_id in range(data.shape[0]): 
        img=[]
        for k in range(start, end, bin_width):
            temp = data_in[trial_id, k:k+bin_width].mean(0)

            temp *= mask2
            img.append(temp)

        img_all.append(np.hstack(img))


    img = np.var(img_all, axis=0)


    vmin = -0.0015
    vmax = 0.0015


    cbar = plt.imshow(img, 
                      vmin=vmin, 
                      vmax=vmax,
               #aspect='auto',
               interpolation='none')
    
    variances.append(img)
    plt.xticks([])
    plt.yticks([])
    #plt.colorbar()

    vmin = np.nanmin(img)
    vmax = np.nanmax(img)
    print ("vmin, vmax: ", vmin, vmax)

    plt.savefig('/home/cat/var_'+str(ctr)+'.svg',dpi=1200)
    plt.close()
    
    
diffs = variances[1]-variances[0]
vmin = -0.0015
vmax = 0.0015

plt.imshow(diffs, 
          vmin=vmin, 
          vmax=vmax,
           #aspect='auto',
           interpolation='none')

variances.append(img)
plt.xticks([])
plt.yticks([])
#plt.colorbar()

plt.savefig('/home/cat/diff.svg',dpi=1200)
plt.close()

[[  0   0]
 [  0   1]
 [  0   2]
 ...
 [127 125]
 [127 126]
 [127 127]]
vmin, vmax:  2.5242667198652134e-06 0.0020088094848655954
vmin, vmax:  1.2770538968371787e-05 0.0020741081641278483
